<a href="https://colab.research.google.com/github/littleShaniZ/ControlNet/blob/SelectiveControlNet/BasicControlNetLineart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Basic ControlNet Lineart Inference (Minimal Setup)

# --- 🧩 Cell 1: Install required packages ---
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q opencv-python transformers diffusers

# --- 🧩 Cell 2: Clone ControlNet v1.1-nightly ---
!git clone https://github.com/lllyasviel/ControlNet-v1-1-nightly.git
%cd ControlNet-v1-1-nightly

# --- 🧩 Cell 3: Download pretrained lineart model ---
!mkdir -p models
!wget -O models/control_sd15_lineart.pth https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_sd15_lineart.pth

# --- 🧩 Cell 4: Import dependencies ---
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import requests
from io import BytesIO
from cldm.model import create_model, load_state_dict
from cldm.ddim_hacked import DDIMSampler
from annotator.lineart import apply_lineart

# --- 🧩 Cell 5: Load and preprocess image ---
image_path = "test_imgs/person_2.png"
input_image = Image.open(image_path).convert("RGB").resize((512, 512))
input_np = np.array(input_image)
hint_np = apply_lineart(input_np)

# --- 🧩 Cell 6: Setup model ---
model = create_model('./configs/controlnet.yaml').cpu()
model.load_state_dict(load_state_dict('./models/control_sd15_lineart.pth', location='cuda'))
model = model.cuda()
model.eval()
sampler = DDIMSampler(model)

# --- 🧩 Cell 7: Prompt setup ---
prompt = "a woman in futuristic cyberpunk style"
n_prompt = "blurry, distorted, low quality"

# --- 🧩 Cell 8: Prepare conditioning input ---
cond = torch.tensor(hint_np / 255.0).float()
cond = cond[None, None, :, :].repeat(1, 3, 1, 1).cuda()

# --- 🧩 Cell 9: Sampling ---
ddim_steps = 30
strength = 1.0
guide_scale = 9.0
eta = 0.0

shape = (4, 64, 64)
uc = model.get_learned_conditioning([n_prompt])
c = model.get_learned_conditioning([prompt])
cond_dict = {"c_concat": [cond], "c_crossattn": [c]}
uc_dict = {"c_concat": [cond], "c_crossattn": [uc]}

model.control_scales = [strength] * 13

samples, _ = sampler.sample(
    ddim_steps, batch_size=1, shape=shape, conditioning=cond_dict,
    verbose=False, unconditional_guidance_scale=guide_scale,
    unconditional_conditioning=uc_dict, eta=eta, x_T=None
)

# --- 🧩 Cell 10: Decode and display result ---
from cldm.model import torch_to_numpy, autoencoder
result = autoencoder.decode(samples[0].unsqueeze(0))
result_image = Image.fromarray(torch_to_numpy(result))

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.title("Input Image")
plt.imshow(input_image)
plt.axis("off")

plt.subplot(1, 3, 2)
plt.title("Lineart Hint")
plt.imshow(hint_np, cmap='gray')
plt.axis("off")

plt.subplot(1, 3, 3)
plt.title("Generated Output")
plt.imshow(result_image)
plt.axis("off")
plt.show()
